In [1]:
import numpy as np
import pandas as pd
from pyarrow import parquet

In [2]:
def gen_col_name_dict(
    curr_names: list[str],
    new_names: list[str]
) -> dict[str: str]:
    """Maps raw data column tags to readable titles.

    Args:
        curr_names: The tags identifying data subset of interest.
        new_names: List of names to replace tags as column headers.

    Returns: Dictionary
    """
    col_names_dict = {
        "NAME": "State",
        "S1501_C01_006E": "Total Pop.",
        "S1501_C01_006M": "Total Pop. ME",
    }
    # Creates a duplicate for each name distinguished by a suffix.
    new_names = [item for name in new_names for item in (name, f"{name} ME")]
    col_names_dict |= {k: v for k, v in zip(curr_names[3:], new_names)}
    return col_names_dict

def cols_to_keep(year: int) -> list[str]:
    """Gets column unique tags of interest.

    Args:
        year: Year is necessary due to certain data being retagged after 2014.

    Returns: The column tags used to select a subset of the raw data.
    """
    base = ["NAME", "S1501_C01_006E", "S1501_C01_006M"]
    adjusted = [
        f"S1501_C0{(year > 2014) + 1}_0{str(n).zfill(2)}{suffix}"
        for n in range(7, 14)
        for suffix in ["E", "M"]
    ]
    return base + adjusted

def gen_edu_df(year: int) -> pd.DataFrame:
    """Generates dataframe from raw educational attainment csv data.
    Contains state-level percentages (and associated margin of error) for
    each ordinal education category of age group 25 and over.

    Args:
        year: Ranges from 2010-2021 excluding 2020 (due to quality issues
            stemming from the covid pandemic).

    Returns: Trimmed dataset.
    """
    path = (
        f"../raw-data/ACSSTEducationalAttainment2010-2021"
        f"/ACSST1Y{year}.S1501-Data.csv"
    )
    new_names = [
        "< 9th Grade", "9-12, no diploma", "HS graduate (or equivalent)",
        "Some college, no degree", "Associate's degree", "Bachelor's degree",
        "Graduate or professional degree",
    ]
    cols = cols_to_keep(year)
    edu_df = (
        pd
        .read_csv(path, usecols=cols, skiprows=[1], dtype={"NAME": "category"})
        .rename(columns=gen_col_name_dict(cols, new_names))
    )
    return edu_df

In [3]:
def gen_ins_df(year: int) -> pd.DataFrame:
    """Generates dataframe from raw insurance coverage csv data.
    Contains state-level percentage (including margin of error) of
    the uninsured for ages 18-64 (filtering was later revised to 19-64
    starting in 2017).
    
    Args:
        year: Ranges from 2010-2021 excluding 2020 (due to quality issues
            stemming from the covid pandemic).

    Returns: Trimmed dataset.
    """
    path = (
        f"../raw-data/ACSSTInsuranceCoverage2010-2021"
        f"/ACSST1Y{year}.S2701-Data.csv"
    )
    if year > 2017:
        cidx, ridx = 5, 12
    elif year > 2014:
        cidx, ridx = 5, 5
    else:
        cidx, ridx = 3, 3
    adjusted = [
        f"S2701_C0{cidx}_0{str(ridx).zfill(2)}{suffix}"
        for suffix in ["E", "M"]
    ]
    cols = ["NAME"] + adjusted
    col_names = ["State", "% Uninsured", "% Uninsured ME"]
    col_names_dict = {k: v for k, v in zip(cols, col_names)}
    inscov_df = (
        pd
        .read_csv(path, usecols=cols, skiprows=[1], dtype={"NAME": "category"})
        .rename(columns=col_names_dict)
    )
    return inscov_df

In [4]:
def gen_pov_df(year: int) -> pd.DataFrame:
    """Generates dataframe from raw poverty csv data.
    Contains state-level percentage (including margin of error) of
    those beneath the poverty line of ages ranging 18-64.

    Args:
        year: Ranges from 2010-2021 excluding 2020 (due to quality issues
            stemming from the covid pandemic).

    Returns: Trimmed dataset.
    """
    path = f"../raw-data/ACSSTPoverty2010-2021/ACSST1Y{year}.S1701-Data.csv"
    adjusted = [
        f"S1701_C03_00{(year > 2014) * 2 + 4}{suffix}" for suffix in ["E", "M"]
    ]
    cols = ["NAME"] + adjusted
    col_names = ["State", "% below poverty", "% below poverty ME"]
    col_names_dict = {k: v for k, v in zip(cols, col_names)}
    poverty_df = (
        pd
        .read_csv(path, usecols=cols, skiprows=[1], dtype={"NAME": "category"})
        .rename(columns=col_names_dict)
    )
    return poverty_df

In [261]:
# def edu_attainment_donut(edu_df, year, state):
#     """
#     """
#     data = edu_df.query(f"State == '{state}'")[new_names].values[0]
#     plt.pie(data, labels=new_names, wedgeprops={'linewidth': 1.5, 'edgecolor': 'white'})
#     plt.gcf().gca().add_artist(plt.Circle((0, 0), 0.65, color='white'))
#     plt.title(f"{state}: {year}")
#     plt.show()

# years = [n for n in range(2010, 2022) if n != 2020]
# #for year in years:
#     #edu_attainment_donut(year, "California")

In [5]:
def gen_rand_df() -> pd.DataFrame:
    """Generates RAND dataframe of select features from raw excel data.
    State-level estimated firearm ownership percentage
    (including standard error). Also contains binary columns signifying
    state universal background checks and permit policies.
    Lastly, restricts year range to that similar of the other datasets.
    Ranges from 2000-2016.

    Returns: Trimmed dataset.
    """
    path = (
        "../raw-data/RAND_TL354.database/"
        "TL-354-State-Level Estimates of Household Firearm Ownership.xlsx"
    )
    cols = ("Year", "STATE", "HFR", "HFR_se", "universl", "permit")
    dtype_dict = {
        "Year": np.uint16,
        "STATE": "category",
        "universl": bool,
        "permit": bool,
    }
    rand_df = pd.read_excel(
        path,
        sheet_name=1,
        usecols=cols,
        dtype=dtype_dict,
    ).query("Year >= 2000").rename(columns={"STATE": "State"})
    return rand_df

#filtered_df = rand_df.query("Year >= 2010 & STATE in ['Missouri', 'Iowa']").copy()
#filtered_df["STATE"] = filtered_df["STATE"].cat.remove_unused_categories()
#sns.lineplot(filtered_df, x="Year", y="HFR", hue="STATE")
#plt.show()

In [6]:
def gen_nsduh_df(year: int) -> pd.DataFrame:
    """Generates merged dataframe based off of two tables from NSDUH raw data.
    The two datasets in question are as followed:
        "Needing But Not Receiving Treatment at a Specialty Facility 
            for Substance Use in the Past Year"
        "Serious Mental Illness in the Past Year"
    Both labeled as percentages and for ages 18 or older.

    Args:
        year: Year is necessary due to reorganization of the
            excel sheet tables. There are two such instances,
            one in 2017 and another in 2019. Ranges from 2016-2019.

    Returns: A multiindex dataframe.
    """
    path = "../raw-data/NSDUH2016-2019/"
    ext = f"NSDUHsaeExcelTabs{year}.xlsx"
    if year < 2017:
        sheets = [25, 26]
        path += ext
    elif year < 2019:
        sheets = [26, 27]
        path += ext
    else:
        sheets = [26, 28]
        path += f"{year}NSDUHsaeExcelTabs.xlsx"
    cols = [
        "State", "18 or Older\nEstimate", "18 or Older\n95% CI (Lower)",
        "18 or Older\n95% CI (Upper)"
    ]
    df_dict = pd.read_excel(path,
        sheet_name=sheets,
        usecols=cols,
        skiprows=[n for n in range(11) if n != 5],
        dtype={"State": "category"}
    )
    sub_use_df, m_illness_df = df_dict.values()
    combined = sub_use_df.merge(m_illness_df, on="State")
    state_multidx = pd.MultiIndex.from_arrays([cols[:1], [""]])
    table_multidx = pd.MultiIndex.from_product(
        [['Needing Treatment for Substance Use', 'Serious Mental Illness'],
        cols[1:]]
    )
    combined.columns = state_multidx.append(table_multidx)
    return combined

In [7]:
# NOTE RAW XLSX WILL NOT BE INCLUDED TO RESPECT THE CREATOR'S WISHES.
# REFINED FILE WILL REMAIN FOR ACCESS, AS IT IS MUCH MORE LIMITED IN SCOPE.
def gen_shootings_df() -> pd.DataFrame:
    """Generates dataframe from select features of the full school shootings
    database. Contains dates and exact coordinate locations for visualization
    purposes and statistics on casualties. Also includes motive of attack
    via the Target column. This information is used to lightly filter
    dataset further by excluding the "neither" category which is often tied
    to accidental or self-injury related incidents. Further filtering
    is done by restricing the time interval similar to the other datasets.

    Returns: Trimmed dataset.
    """
    path = (
        "../raw-data/Public v3.1 K-12 School Shooting Database "
        "(April 17 2023 with GIS).xlsx"
    )
    cols = (
        "Date", "Victims_Killed", "Victims_Wounded",
        "Number_Victims", "Targets", "State", "LAT", "LNG"
    )
    shootings_df = pd.read_excel(
        path,
        sheet_name=1,
        usecols=cols,
        dtype={
            "Victims_Killed": np.uint16,
            "Victims_Wounded": np.uint16,
            "Number_Victims": np.uint16,
            "State": "category",
            "Targets": "category",
        }
    ).query("Targets != 'Neither' and Date >= '2000-01-01'")
    return shootings_df

In [8]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [9]:
def export_preprocessing() -> tuple[pd.DataFrame]:
    """Processes dataframes for exporting. Degree of processing varies
    depending on raw data. Dataframes generated on individual year basis
    are combined into a single full time range dataframe. ACSST dataframes
    (edu, ins, pov) are also merged due to sharing same states and timespans.
    Each dataframe then has its state column converted to state codes for
    ease of use in plotly.

    Returns: All processed dataframes as a tuple. Note that second element
        (nsduh_df) is unique due to being the only multiindex df.
    """
    full_dfs = ([], [], [], [])
    years = np.array([year for year in range(2010, 2022)], dtype=np.uint16)
    # Stores all yearly chunks into individual lists for each per year dataset.
    for year in years:
        if year != 2020:
            df_lst = [gen_edu_df(year), gen_ins_df(year), gen_pov_df(year)]
            for i, df in enumerate(df_lst):
                df["Year"] = year
                full_dfs[i].append(df)
        if 2015 < year < 2020:
            nsduh_df = gen_nsduh_df(year)
            nsduh_df["Year"] = year
            full_dfs[len(df_lst)].append(nsduh_df)
    # Concat yearly dfs into single full time range df
    # for each per year dataset collection.
    concated_lst = []
    for df_sublst in full_dfs:
        concated_lst.append(pd.concat(df_sublst))
    acsst_df = concated_lst[0]
    # Chain merge all ACSST dfs into a single master df.
    for i in range(1, len(concated_lst) - 1):
        acsst_df = acsst_df.merge(concated_lst[i], on=["Year", "State"])
    for df in [acsst_df, concated_lst[-1], rand_df := gen_rand_df()]:
        df["State"] = df["State"].cat.rename_categories(us_state_to_abbrev)
    return acsst_df, concated_lst[-1], rand_df, gen_shootings_df()

def export_refined() -> None:
    """Saves final Dataframes to be used in data analysis.
    """
    out = [f"refined-{name}" for name in ["ACSST", "NSDUH", "RAND", "SSDB"]]
    for df, path in zip(export_preprocessing(), out):
        df.to_parquet(f"../refined-data/{path}.parquet")

export_refined()